this jupyter notebook is only used to test the model after training it with the other jupyter notebook.

In [ ]:
# installig usefull libraries
!pip install googledrivedownloader
!pip install tldextract
!pip install transformers

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
# Downloading library
gdd.download_file_from_google_drive(file_id='1E5tch0iYph3cGDSXOLXmqEnRb4mml1wV',
                                    dest_path='./tools.py',
                                    )
# Connecting to colab and get files
from google.colab import drive ,files
import torch
import numpy as np
from transformers import AutoTokenizer
from tools import *

In [ ]:
# Connecting to drive to save the dataloaders
drive.mount('/content/drive')
# Getting label encoder from drive
label_encoder = LabelEncoder()
label_encoder.load('drive/MyDrive/label_encoder')
model = torch.load('drive/MyDrive/camemBert_model_URLs',map_location=torch.device('cpu'))

Mounted at /content/drive


In [ ]:
URLs = ['https://www.logifac.fr/residence/la-residence-gondoles-choisy-le-roi/','https://www.researchgate.net/publication/352563832_MULTILABEL_OVER-SAMPLING_AND_UNDER-SAMPLING_WITH_CLASS_ALIGNMENT_FOR_IMBALANCED_MULTILABEL_TEXT_CLASSIFICATION']

In [ ]:
# preprocessing urls
preprocessed_data = [preprocess_url(url,final_stopwords_list) for url in URLs]

max_length = 30
# getting the tokenizer
tokenizer = AutoTokenizer.from_pretrained("camembert-base", do_lower_case=True)
# tokenizing the urls
preprocessed_data = tokenizer.batch_encode_plus(preprocessed_data,max_length=max_length,pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# doing the inferencce and geting the logits
out = model(torch.tensor(preprocessed_data['input_ids']), token_type_ids=None, attention_mask=torch.tensor(preprocessed_data['attention_mask']))[0]
pred_label = torch.sigmoid(out).detach().numpy()

threshold = 0.50
# conveting the output to binray vectors
pred_bools = np.where(pred_label>threshold,1,0)

In [ ]:
labels = label_encoder.decode(pred_bools)
print("the labels are: ",labels)

the labels are:  [['1057', '1077', '108', '1094', '1107', '1111', '1146', '115', '1193', '1234', '1265', '1292', '1343', '1367', '1368', '1369', '1370', '1372', '1410', '1494', '1526', '1533', '1556', '1573', '1687', '1690', '1722', '1730', '185', '191', '210', '270', '294', '317', '329', '34', '353', '365', '374', '474', '5186', '529', '540', '5529', '5692', '5693', '5697', '572', '582', '608', '617', '63', '676', '784', '852', '89', '906', '907', '908', '920', '925', '96', '966', '978', '992', '997'], ['1057', '1077', '108', '1094', '1107', '1111', '1143', '1146', '115', '1163', '1193', '1234', '1265', '1292', '1343', '1367', '1368', '1369', '1370', '1372', '1494', '1526', '1533', '1534', '1556', '1573', '1687', '1690', '1722', '1730', '1781', '185', '191', '210', '294', '317', '329', '34', '353', '365', '374', '377', '401', '474', '529', '540', '5529', '5692', '5693', '5697', '572', '582', '608', '617', '63', '676', '784', '852', '89', '906', '907', '908', '920', '925', '937', '96',